In [1]:
import sqlite3
import pandas as pd

# Create your connection.
# DL : https://drive.google.com/file/d/10CtW_gZtNHQbQQ2A7QRiXXTh4obXHOQk/view?usp=sharing
con = sqlite3.connect('batch_1.db')
cursorObj = con.cursor()
cursorObj.execute('SELECT name from sqlite_master where type= "table"')
print(cursorObj.fetchall())

[('aeroports',), ('compagnies',), ('vols',)]


In [2]:
aeroports = pd.read_sql_query("SELECT * FROM aeroports", con)
aeroports.to_csv("aeroports.csv", index=False)

In [3]:
compagnies = pd.read_sql_query("SELECT * FROM compagnies", con)
compagnies.to_csv("compagnies.csv", index=False)

In [4]:
vols = pd.read_sql_query("SELECT * FROM vols", con)
vols.to_csv("vols.csv", index=False)

In [13]:
airports = aeroports.drop_duplicates(subset=['CODE IATA'])

airports_start = airports.add_prefix('air_dep_')
airports_end = airports.add_prefix('air_arr_')

# Add companies
vols_and_companies = vols.join(compagnies.set_index(['CODE']),
                               on=['COMPAGNIE AERIENNE'],
                               how='left')

# Add airport departure information
vols_and_companies_departure = vols_and_companies.join(airports_start.set_index(
    ['air_dep_CODE IATA']),
    on=['AEROPORT DEPART'],
    how='inner')

# Add airport arrival information
full_df = vols_and_companies_departure.join(airports_end.set_index(
    ['air_arr_CODE IATA']),
    on=['AEROPORT DEPART'],
    how='inner')

vols.to_csv("full_join.csv", index=False)